# Convolutional neural network

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [7]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../Download/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../Download/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
shuffle=False)

In [8]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [9]:
model = ConvNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1467
Epoch [1/5], Step [200/600], Loss: 0.1326
Epoch [1/5], Step [300/600], Loss: 0.0690
Epoch [1/5], Step [400/600], Loss: 0.0427
Epoch [1/5], Step [500/600], Loss: 0.0270
Epoch [1/5], Step [600/600], Loss: 0.1167
Epoch [2/5], Step [100/600], Loss: 0.0592
Epoch [2/5], Step [200/600], Loss: 0.0258
Epoch [2/5], Step [300/600], Loss: 0.0330
Epoch [2/5], Step [400/600], Loss: 0.0911
Epoch [2/5], Step [500/600], Loss: 0.0764
Epoch [2/5], Step [600/600], Loss: 0.0646
Epoch [3/5], Step [100/600], Loss: 0.0157
Epoch [3/5], Step [200/600], Loss: 0.0288
Epoch [3/5], Step [300/600], Loss: 0.0159
Epoch [3/5], Step [400/600], Loss: 0.0484
Epoch [3/5], Step [500/600], Loss: 0.0283
Epoch [3/5], Step [600/600], Loss: 0.0504
Epoch [4/5], Step [100/600], Loss: 0.0321
Epoch [4/5], Step [200/600], Loss: 0.0026
Epoch [4/5], Step [300/600], Loss: 0.0539
Epoch [4/5], Step [400/600], Loss: 0.0088
Epoch [4/5], Step [500/600], Loss: 0.0020
Epoch [4/5], Step [600/600], Loss:

In [11]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 99.04 %


In [12]:
torch.save(model.state_dict(), 'model.ckpt')